# 🍎 Chat Completions with AIProjectClient 🍏

In this notebook, we'll demonstrate how to perform **Chat Completions** using the **Azure AI Foundry** SDK. We'll combine **`azure-ai-projects`** and **`azure-ai-inference`** packages to:

1. **Initialize** an `AIProjectClient`.
2. **Obtain** a Chat Completions client to do direct LLM calls.
3. **Use** a **prompt template** to add system context.
4. **Send** user prompts in a health & fitness theme.

## 🏋️ Health-Fitness Disclaimer
> **This example is for demonstration only and does not provide real medical advice.** Always consult a professional for health or medical-related questions.

### Prerequisites
Before starting this notebook, please ensure you have completed all prerequisites listed in the root [README.md](../../README.md#-prerequisites).

Let's get started! 🎉

<img src="./seq-diagrams/1-chat.png" width="30%"/>


In [9]:
import os
import json
from pathlib import Path

def find_cred_json(start_path):
    # Start from current directory and go up
    current = Path(start_path)
    while current != current.parent:  # while we haven't hit the root
        cred_file = current / 'cred.json'
        if cred_file.exists():
            return str(cred_file)
        current = current.parent
    return None

# Search in the parent directory and its subdirectories
parent_dir = os.path.dirname(os.getcwd())  # Get parent directory
file_path = find_cred_json( parent_dir)

print(f"Found cred.json at: {file_path}")

try:
    with open(file_path, 'r') as f:
        loaded_config = json.load(f)
    
    # Print the values to verify
    print("Project Connection String:", loaded_config['PROJECT_CONNECTION_STRING'])
    print("Tenant ID:", loaded_config['TENANT_ID'])
    print("Model Deployment ID:", loaded_config['MODEL_DEPLOYMENT_NAME'])
    
except FileNotFoundError:
    print(f"Could not find file at: {file_path}")
except json.JSONDecodeError:
    print(f"File exists but contains invalid JSON")


Found cred.json at: d:\MLOps\Gen Ai & MLOps Masterclass\Materilas\test\ai-foundry-workshop\cred.json
Project Connection String: eastus2.api.azureml.ms;1c2fd79b-ad21-4ad0-8d53-12de16650452;rg-sarath-4339_ai;sarath-1143
Tenant ID: 02e58275-def8-41c4-82c4-f7864c28f7c9
Model Deployment ID: gpt-4o


## 1. Initial Setup
Load environment variables, create an `AIProjectClient`, and fetch a `ChatCompletionsClient`. We'll also define a **prompt template** to show how you might structure a system message.


In [10]:
import os
from dotenv import load_dotenv
from pathlib import Path
from azure.identity import DefaultAzureCredential

from azure.ai.projects import AIProjectClient
from azure.ai.inference.models import UserMessage, SystemMessage  # for chat messages

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Retrieve from environment
connection_string = loaded_config["PROJECT_CONNECTION_STRING"]
model_deployment = loaded_config["MODEL_DEPLOYMENT_NAME"]

try:
    # Create the project client
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=connection_string,
    )
    print("✅ Successfully created AIProjectClient")
except Exception as e:
    print("❌ Error initializing client:", e)


✅ Successfully created AIProjectClient


### Prompt Template
We'll define a quick **system** message that sets the context as a friendly, disclaimer-providing fitness assistant.

```txt
SYSTEM PROMPT (template):
You are FitChat GPT, a helpful fitness assistant.
Always remind users: I'm not a medical professional.
Be friendly, provide general advice.
...
```

We'll then pass user content as a **user** message.


In [11]:
# We'll define a function that runs chat completions with a system prompt & user prompt
def chat_with_fitness_assistant(user_input: str):
    """Use chat completions to get a response from our LLM, with system instructions."""
    # Our system message template
    system_text = (
        "You are FitChat GPT, a friendly fitness assistant.\n"
        "Always remind users: I'm not a medical professional.\n"
        "Answer with empathy and disclaimers."
    )

    # We'll open the chat completions client
    with project_client.inference.get_chat_completions_client() as chat_client:
        # Construct messages: system + user
        system_message = SystemMessage(content=system_text)
        user_message = UserMessage(content=user_input)

        # Send the request
        response = chat_client.complete(
            model=model_deployment,
            messages=[system_message, user_message]
        )

        return response.choices[0].message.content  # simplest approach: get top choice's content

print("Defined a helper function to do chat completions.")

Defined a helper function to do chat completions.


## 2. Try Chat Completions 🎉
We'll call the function with a user question about health or fitness, and see the result. Feel free to modify the question or run multiple times!


In [12]:
user_question = "How can I start a beginner workout routine at home?"
reply = chat_with_fitness_assistant(user_question)
print("🗣️ User:", user_question)
print("🤖 Assistant:", reply)

🗣️ User: How can I start a beginner workout routine at home?
🤖 Assistant: Starting a workout routine at home is a fantastic idea—congratulations on taking the first step toward your fitness journey! 💪❤️ Remember, I'm not a medical professional, so if you have any underlying health conditions or concerns, consult a doctor or a certified fitness trainer before beginning.

That said, here’s how you can get started safely and effectively:

---

### **Step 1: Set Realistic Goals**
Ask yourself why you’re working out (e.g., to feel stronger, improve energy, or stay healthy). Setting clear, achievable goals will help you stay motivated.

---

### **Step 2: Create a Simple Plan**
For beginners, consistency matters more than intensity. Start with short, manageable workouts (15–30 minutes, 2–4 times per week) and build from there.

Here’s a sample format you can follow:

#### **Warm-Up (5–10 minutes)**
Prepare your body by increasing circulation and loosening your joints:
- March in place
- Arm 

## 3. Another Example: Prompt Template with Fill-Ins 📝
We can go a bit further and add placeholders in the system message. For instance, imagine we have a **userName** or **goal**. We'll show a minimal example.


In [13]:
def chat_with_template(user_input: str, user_name: str, goal: str):
    # Construct a system template with placeholders
    system_template = (
        "You are FitChat GPT, an AI personal trainer for {name}.\n"
        "Your user wants to achieve: {goal}.\n"
        "Remind them you're not a medical professional. Offer friendly advice."
    )

    # Fill in placeholders
    system_prompt = system_template.format(name=user_name, goal=goal)

    with project_client.inference.get_chat_completions_client() as chat_client:
        system_msg = SystemMessage(content=system_prompt)
        user_msg = UserMessage(content=user_input)

        response = chat_client.complete(
            model=model_deployment,
            messages=[system_msg, user_msg]
        )

    return response.choices[0].message.content

# Let's try it out
templated_user_input = "What kind of home exercise do you recommend for a busy schedule?"
assistant_reply = chat_with_template(
    templated_user_input,
    user_name="Jordan",
    goal="increase muscle tone and endurance"
)
print("🗣️ User:", templated_user_input)
print("🤖 Assistant:", assistant_reply)

🗣️ User: What kind of home exercise do you recommend for a busy schedule?
🤖 Assistant: Got it, Jordan! Let’s optimize your time with some efficient home exercises that’ll help you increase muscle tone and endurance. Reminder: I'm not a medical professional, so make sure to listen to your body and consult a healthcare provider if needed before starting something new. 💪

Here’s a quick and effective workout you can do in 20–30 minutes at home:

---

### **Dynamic Warm-Up (3–5 Minutes)**  
1. **Jumping Jacks** – 1 minute to elevate your heart rate.  
2. **Arm Circles** – 30 seconds forward, 30 seconds backward.  
3. **Bodyweight Good Mornings** – 10 reps to warm up your hamstrings and lower back.

---

### **Full-Body Circuit (20 Minutes)**  
Perform each exercise for 40 seconds, then rest for 20 seconds. Repeat the entire circuit 3–4 times. Adjust the number of rounds based on your time and energy.

1. **Push-Ups** – Builds upper-body strength and tones your chest, shoulders, and arms. M

## 🎉 Congratulations!
You've successfully performed **chat completions** with the Azure AI Foundry's `AIProjectClient` and `azure-ai-inference`. You've also seen how to incorporate **prompt templates** to tailor your system instructions.

#### Head to [2-embeddings.ipynb](2-embeddings.ipynb) for the next part of the workshop! 🎯